In [ ]:
!pip install -r requirements.txt

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from games.tileman.envs.multi_agent_env import TileServer, ClientPlayerEnv

In [ ]:
proc = TileServer.start_popen_process()

In [ ]:
proc.terminate()

In [ ]:
env = ClientPlayerEnv()
env.reset()

while True:
    obs, reward, done, info = print(env.step(0))
    if done:
        env.reset()

In [2]:
import nest_asyncio
nest_asyncio.apply()
import pygame
from stable_baselines3.common.env_util import make_vec_env
from games.tileman.envs.multi_agent_env import TileServer, ClientPlayerEnv
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.evaluation import evaluate_policy

vec_env = make_vec_env("tileman-multi-v0", n_envs=8, env_kwargs=dict(vision_range=5))

In [ ]:
model = PPO("MlpPolicy", vec_env, verbose=0, n_steps=5000, batch_size=100, n_epochs=8, learning_rate=0.0002, policy_kwargs=dict(net_arch=[416, 416]))

class SaveEvalCallback(BaseCallback):
    def __init__(self, eval_freq=10000, verbose=0):
        super(SaveEvalCallback, self).__init__(verbose)
        self.eval_env = make_vec_env("tileman-multi-v0", n_envs=1, env_kwargs=dict(vision_range=5))
        self.eval_freq = eval_freq
        self.best_mean_reward = -float('inf')

    def _on_step(self) -> bool:
        if self.n_calls % self.eval_freq == 0:
            reward, _ = evaluate_policy(
                model.policy,  # type: ignore[arg-type]
                vec_env,
                n_eval_episodes=3,
                render=False,  # comment out to speed up
            )
            if self.verbose > 0:
                print(f"Step: {self.n_calls}, Reward: {reward}")
            if reward > self.best_mean_reward:
                self.best_mean_reward = reward
                self.model.save(f"best_models/{self.n_calls}/{reward}")
        return True
    
model.learn(total_timesteps=100_000_000, progress_bar=True, callback=SaveEvalCallback(eval_freq=10000, verbose=1))
obs = vec_env.reset()

TypeError: __init__() got an unexpected keyword argument 'grid_size' was raised from the environment creator for tileman-multi-v0 with kwargs ({'grid_size': 40, 'vision_range': 5})

In [ ]:
model.save("test")